In [ ]:
PROMPT = """
You are a professional machine learning conference reviewer who reviews a given paper and considers 4 criteria: **importance and novelty**, **potential reasons for acceptance**, **potential reasons for rejection**, and **suggestions for improvement**.

You just need to use the following JSON format for output, but don't output opinions that don't exist in the original reviews. if you're not sure, return an empty dict:

{
'Significance and novelty': List multiple items by using Dict, The key is a brief description of the item, and the value is a detailed description of the item.

'Potential reasons for acceptance': List multiple items by using Dict, The key is a brief description of the item, and the value is a detailed description of the item.

"Potential reasons for rejection": List multiple items by using Dict, The key is a brief description of the item, and the value is a detailed description of the item.

'Suggestions for improvement': List multiple items by using Dict, The key is a brief description of the item, and the value is a detailed description of the item.
}

The given paper is as follows.:
{Paper}
"""